In [1]:
import os
import numpy as np
import nibabel as nib
import torch
import matplotlib.pyplot as plt
import glob
from dataloader import SkullDataset
from train import learn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import torch.nn as nn

In [2]:
import segmentation_models_pytorch as smp

model = smp.Unet(
    encoder_name="resnet18",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pretrained weights for encoder initialization
    in_channels=1,                  # model input channels (1 for grayscale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)

In [3]:
label_regex = "NFBS_Dataset/*/*brainmask*"
data_regex = "NFBS_Dataset/*/*T1w.nii.gz"

In [4]:
train_dataset = SkullDataset(data_regex=data_regex,label_regex=label_regex,visualize=False,train=True)
val_dataset = SkullDataset(data_regex=data_regex,label_regex=label_regex,visualize=False,train=False)

In [5]:
train_dataloader = DataLoader(train_dataset, batch_size=64,
                        shuffle=True, num_workers=6, pin_memory=True)
val_dataloader = DataLoader(val_dataset, batch_size=64,
                        shuffle=False, num_workers=6, pin_memory=True)

In [6]:
optimizer = Adam([{'params': model.encoder.parameters(), 'lr': 1e-5},
                {'params': model.decoder.parameters(), 'lr': 1e-4}])
loss_function = nn.BCEWithLogitsLoss()
epochs = 20
device = "cuda:0"
model.to(device)
learn(
    train_dataloader,
    val_dataloader,
    model,
    loss_function,
    optimizer,
    device,
    epochs,
    scheduler=None,
)

  0%|          | 0/20 [00:00<?, ?it/s]


 Epoch 1 / 20



0it [00:36, ?it/s]
  0%|          | 0/20 [00:36<?, ?it/s]


KeyboardInterrupt: 